In [41]:
import geopandas as gpd
import os

# Fix projection celestial body mismatch
os.environ['PROJ_IGNORE_CELESTIAL_BODY'] = 'YES'

geo_units = gpd.read_file('./SIM3292_MarsGlobalGeologicGIS_20M/SIM3292_Shapefiles/SIM3292_Global_Geology.shp')
geo_structures = gpd.read_file('./SIM3292_MarsGlobalGeologicGIS_20M/SIM3292_Shapefiles/SIM3292_Global_Structure.shp')

geo_units = geo_units.to_crs(4326)
geo_structures = geo_structures.to_crs(4326)

geo_units["geometry"] = geo_units.geometry.simplify(0.1)
geo_structures["geometry"] = geo_structures.geometry.simplify(0.1)

unique_units = geo_units["UnitDesc"].dropna().unique()
unique_structures = geo_structures["Origin"].dropna().unique()

In [84]:
from arcgis.gis import GIS
import arcgis.map as map
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcgis.layers

gis = GIS("home")

In [134]:
# IN MEMORY STORAGE
units_sdf = GeoAccessor.from_geodataframe(geo_units).spatial.to_feature_collection()
structures_sdf = GeoAccessor.from_geodataframe(geo_structures).spatial.to_feature_collection()

# UPLOAD TO AGOL AS FEATURE LAYERS
units_layer = GeoAccessor.from_geodataframe(geo_units).spatial.to_featurelayer()
structures_layer = GeoAccessor.from_geodataframe(geo_structures).spatial.to_featurelayer()

c:\Users\trank\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-2\Lib\site-packages\pandas\core\dtypes\cast.py:1066: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\trank\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-2\Lib\site-packages\pandas\core\dtypes\cast.py:1091: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\trank\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-2\Lib\site-packages\pandas\core\dtypes\cast.py:1066: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\trank\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-2\Lib\site-packages\pandas\core\dtypes\cast.py:1091: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\trank\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-2\Lib\concurrent\futures\thread.py:58: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\Users\\trank\\A

In [122]:
marsMap = map.Map()

marsMap.center = [137.35, -4.7]
marsMap.zoom = 11

In [123]:
mars_tile = {
    "type": "WebTiledLayer",
    "url": "https://astro.arcgis.com/arcgis/rest/services/OnMars/MColorDEM/MapServer/tile/{level}/{row}/{col}",
    "copyright": "USGS"
}

marsMap.content.add(mars_tile)

In [124]:
import random

def rand_color():
    return [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), 100]

unit_colors = {u: rand_color() for u in unique_units}
struct_colors = {s: rand_color() for s in unique_structures}

unit_renderer = {
    "type": "uniqueValue",
    "field1": "UnitDesc",
    "defaultSymbol": {
        "type": "esriSFS",
        "style": "esriSFSSolid",
        "color": [200, 200, 200, 80],
        "outline": {"color":[50, 50, 50, 255], "width": 0.3}
    },
    "uniqueValueInfos": 
        [{
            "value": "{UnitDesc}",
            "label": "{UnitDesc}",
            "symbol": {
                "type": "esriSFS",
                "style": "esriSFSSolid",
                "color": unit_colors[unit],
                "outline": {"color":[0, 0, 0, 50], "width": 0.3}
            }
        }
        for unit in unique_units]
}

struct_renderer = {
    "type": "uniqueValue",
    "field1": "Origin",
    "defaultSymbol": {
        "type": "esriSLS",
        "style": "esriSLSSolid",
        "color": [200, 200, 200, 80],
        "outline": {"color":[50, 50, 50, 255], "width": 0.3}
    },
    "uniqueValueInfos": 
        [{
            "value": "{Origin}",
            "label": "{Origin}",
            "symbol": {
                "type": "esriSLS",
                "style": "esriSLSSolid",
                "color": struct_colors[origin],
                "outline": {"color":[0, 0, 0, 50], "width": 0.3}
            }
        }
        for origin in unique_structures]
}

In [125]:
marsMap.content.add(
    structures_sdf,
    {
        "title": "Geologic Structures",
        "renderer": struct_renderer},
    popup_info = {
            "title": "Geological Structure | {Origin}",
            "fieldInfos": [
                {"fieldName": "Morphology", "label": "Morphology", "visible": True},
                {"fieldName": "Origin", "label": "Origin", "visible": True},
                {"fieldName": "Interpreta", "label": "Interpretation", "visible": True},
                {"fieldName": "Preservati", "label": "Preservation", "visible": True},
            ]
        }
)

In [126]:
marsMap.content.add(
    units_sdf,
    {
        "title": "Geological Units",
        "renderer": unit_renderer},
    popup_info = {
            "title": "{UnitDesc}",
            "fieldInfos": [
                {"fieldName": "UnitDesc", "label": "Unit Description", "visible": True}
            ]
        }
)

In [137]:
marsMap

Map(center=[-131.42788760664484, 44.921706907800136], extent={'spatialReference': {'wkid': 102100}, 'xmin': -2…